# Isl Concept

本篇主要介绍`isl`库中的一些概念, 以及一些类与函数的使用方式, 方便大家理解这些操作所代表的意义.

主要是对于[polycomp-tutorial](https://libisl.sourceforge.io/tutorial.pdf), 以及[isl manual](https://libisl.sourceforge.io/manual.pdf)做了一个粗浅总结, 建议大家尽量参考原版文档.


# Values

`isl`中使用`isl.val`表示整数值、有理值或三个特殊值之一，无穷大、负无穷大和`NaN`.可以使用以下函数创建一些预定义值.

In [2]:
import isl

In [2]:

print("val 构造:")

print(isl.val.one())
print(isl.val.zero())
print(isl.val.infty())
print(isl.val(123))
print(isl.val("456"))

print("val 比较:")

print(isl.val.one().is_one())
print(isl.val("1").is_one())
print(isl.val("1").is_zero())
print(isl.val("1").le(isl.val("2")))

print("val 计算:")

print(isl.val(3).mul(isl.val(2)))
print(isl.val(6).is_divisible_by(isl.val(2)))


val 构造:
1
0
infty
123
456
val 比较:
True
True
False
True
val 计算:
6
True


# Identifiers

`Identifier`用于识别单个维度和维度元组.它们由一个可选的`name`和一个可选的`void *user`组成.然而,名称和用户指针不能都是NULL.具有相同`name`但不同的`user`指针值的`id`被认为是不同的.同样,`name`不同但`user`指针值相同的标识符也被认为是不同的.相等的标识符使用相同的对象表示.因此,可以使用==运算符测试标识符对的相等性.可以使用以下函数构建、复制、释放、检查和打印标识符.

⚠️在`python`接口下, 无法随意操作指针, 所以他的`user`指针都是默认值并且会默认自增, 也无法由用户传递/修改. 因此在`python`中检查`id`等价性就少了一条标准.

In [3]:
print("Id 构造:")
a = isl.id("A")
b = isl.id("B")

print("Id 比较:")
print("构造函数内会自增指针指导致不等价:", isl.id("A") == isl.id('A'))
print(a == a)

Id 构造:
Id 比较:
构造函数内会自增指针指导致不等价: False
True


# Space

每当从头开始创建新`set`,`map`或类似对象时,都需要使用`isl_space`指定它所处的空间.每个空间涉及零个或多个参数以及集合或输入/输出维度的零\一个或两个元组.参数和维度由`isl_dim_type`和位置标识.类型`isl_dim_param`指的是参数,类型`isl_dim_set`指的是`set dimensions`(对于具有单个维度元组的空间),类型`isl_dim_in`和`isl_dim_out`指的是输入和输出维度(对于具有两个二元组的空间). `Local Space`还包含`isl_dim_div`类型的维度. 请注意,参数仅通过它们在给定对象中的位置来识别.在不同的对象中,参数(通常)通过其名称或标识符来识别.只有未命名的参数才能通过它们在对象中的位置来识别.不鼓励使用未命名的参数.

In [3]:
print("space 构造:")
unit = isl.space.unit()
print(unit)
print("添加一个参数:", unit.add_param("A"))
print("添加一个0维tuple名字为A:", unit.add_named_tuple("A", 0))
print("添加一个2维tuple名字为B:", unit.add_named_tuple("B", 2))

print("获取space :")
print("set:", isl.set("[n] -> { A[a,b,c] : a < 0 and b> 0 and c >10 }").space())
print("union_set:", isl.union_set("[n] -> { A[a,b,c]; }").space())
print("map:", isl.map("[n] -> { A[a,b,c] -> B[c,a,b] : a > 0; }").space())
print("union_map:", isl.union_map("[n] -> { A[a,b,c] -> B[c,a,b] : a > 0;  B[x] -> C[x] : x < 0;}").space())

print("space 属性:")
a = unit.add_param("A")
print("A的dim类型为IN的个数:", a.dim(isl.ISL_DIM_TYPE.IN))
print("A的dim类型为PARAM的个数:", a.dim(isl.ISL_DIM_TYPE.PARAM))

print("space 修改:")
print("找到dim中param0修改id为B:", a.set_dim_id(isl.ISL_DIM_TYPE.PARAM, 0, isl.id("B")))
sp = isl.space.unit().add_named_tuple("A", 3)
print("构造一个有名字的三个维度的tuple space:", sp)
sp = sp.set_dim_name(isl.ISL_DIM_TYPE.SET, 0, 'i')
print("修改第0个维度的名字:", sp)
print("product两个space:", sp.product(sp))


space 构造:
{  :  }
添加一个参数: [A] -> {  :  }
添加一个0维tuple名字为A: { A[] }
添加一个2维tuple名字为B: { B[i0, i1] }
获取space :
set: [n] -> { A[a, b, c] }
union_set: [n] -> {  :  }
map: [n] -> { A[a, b, c] -> B[o0, o1, o2] }
union_map: [n] -> {  :  }
space 属性:
A的dim类型为IN的个数: 0
A的dim类型为PARAM的个数: 1
space 修改:
找到dim中param0修改id为B: [B] -> {  :  }
构造一个有名字的三个维度的tuple space: { A[i0, i1, i2] }
修改第0个维度的名字: { A[i, i1, i2] }
product两个space: { [A[i, i1, i2] -> A[i', i4, i5]] }


# Sets and Relations

`polycamp`的教程中, 首先介绍了`Named Integer Tuple`, 它由命名整数元组的符号由标识符形成，然后是逗号分隔的整数列表, 比如`A[1,2,5]`. 当不设定名字时可以称为`Unnamed Integer Tuple`,比如`[2,0,4]`.

然后`set`通过花括号中使用分号分隔元素来表示, 比如`{ []; A[1,2] }`. 这在isl中被称为`union set`.


In [5]:
print("set 构造")
print(isl.set("{ A[1,2] }"))
print(isl.set("{ A[a] : a < 10 }"))
print(isl.union_set("{ []; A[1,2] }"))
print(isl.set.empty(isl.space.unit()))
print(isl.set.empty(isl.space.unit().add_named_tuple("a", 1)))

print("set 操作:")
print("intersect :", isl.union_set("{ []; A[1,2] }").intersect(isl.union_set("{ []; A[1] }")))
print("union :", isl.union_set("{ []; A[1,2] }").union(isl.union_set("{ []; C[4] }")))
print("subset :", isl.union_set("{ A[1,2] }").is_subset(isl.union_set("{  A[1,2,4] }")))
print("identity:", isl.set("{A[a] : a > 10}").identity())



set 构造
{ A[1, 2] }
{ A[a] : a <= 9 }
{ []; A[1, 2] }
{  : false }
{ a[i0] : false }
set 操作:
intersect : { [] }
union : { []; C[4]; A[1, 2] }
subset : False
identity: { A[a] -> A[a' = a] : a >= 11 }



- Quantifier Elimination

  Quantifier Elimination是将可能存在量化变量的Presburger公式，重写为等效公式，重写后该公式不涉及任何量化的变量. 通过`compute_divs`来调用.

- Coalescing
  简化整数空间约束, 通过`coalesce`调用.

In [6]:
print("compute_divs :", isl.union_set("{ A[x] : exists a : x < 3a < 2x }").compute_divs()) 
print("coalesce :", isl.union_set("{ A[x] : exists a : x < 3a < 2x }").coalesce()) 
print("coalesce :", isl.union_set("{ A[x] : 1 <=x <= 5 or 6 <= x <= 10 }").coalesce()) 


compute_divs : { A[x] : x >= 2 and 3*floor((-1 - x)/3) > -2x }
coalesce : { A[x] : exists (e0: x < 3e0 < 2x) }
coalesce : { A[x] : 0 < x <= 10 }


## Binary Relations

在isl中使用`->`来表示两个`named integer tuple`是一个二元关系`pair`. 在`isl`中, 这种二元关系通过`union map`来表示. 并且将`pair`的前部分称为`domain`, 后部分称为`range`.

In [7]:

print("map 构造")
print(isl.union_map("{ A[2, 8, 1] -> B[5]; A[2, 8, 1] -> B[6]; B[5] -> B[5] }"))
print(isl.map("{ A[1] -> B[2,3,4]}"))
print("从domain -> range:", isl.union_map.from_domain_and_range(isl.union_set(
    "{ A[2 ,8 ,1] ; B[5]}"), isl.union_set("{ B[5]; B[6] }")))  # 这是product的构造.

print("map 操作")
print("交集:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").intersect(
    isl.union_map(" { A[2 ,8 ,1] -> B[6]; B[5] -> B[5] }")))
print("反向:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").reverse())

print("取前部分:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").range())
print("取后部分:", isl.union_map("{ A[2 ,8 ,1] -> B[5]; B[5] -> B[5] }").domain())


map 构造
{ B[5] -> B[5]; A[2, 8, 1] -> B[6]; A[2, 8, 1] -> B[5] }
{ A[1] -> B[2, 3, 4] }
从domain -> range: { A[2, 8, 1] -> B[6]; A[2, 8, 1] -> B[5]; B[5] -> B[6]; B[5] -> B[5] }
map 操作
交集: { B[5] -> B[5] }
反向: { B[5] -> A[2, 8, 1]; B[5] -> B[5] }
取前部分: { B[5] }
取后部分: { B[5]; A[2, 8, 1] }


# Points

`points`是`set`的元素,通常可以用于构造`set`,或者表示`set`中所有独立的元素.

In [8]:
print("point 构造:")
unit = isl.space.unit()
unit = unit.add_param("A")
print(unit)
zero_point = isl.point.zero(unit)
print(zero_point)
print("获取一个point:", isl.set("{ A[i,j] : 0 <= i <= j < 10}").sample_point())

point 构造:
[A] -> {  :  }
[A = 0] -> {  }
获取一个point: { A[0, 0] }


In [9]:
print("point 操作:")

print("设定参数A为10:", zero_point.add_ui(isl.ISL_DIM_TYPE.PARAM, 0, 10))
isl.set("{ A[i,j] : 0 <= i <= j < 2}").foreach_point(lambda p: print(p))
print("遍历所有的point:", zero_point.set_coordinate_val(isl.ISL_DIM_TYPE.PARAM, 0, isl.val("123")))


point 操作:
设定参数A为10: [A = 10] -> {  }
{ A[0, 0] }
{ A[0, 1] }
{ A[1, 1] }
遍历所有的point: [A = 123] -> {  }


# Functions

除了`set/relation`, `isl`也提供了多种类型的函数. 每个类型都从源自`value`的类型, 或者通过零个或多个的`primitive function`类型之一来构造. 

特殊情况下, 我们也可以通过`id`来构造`multi expression`, 要注意这并不是`function`类型.

接下来我们先介绍`Primitive`类型,然后再介绍衍生类型.

## Primitive Functions

ISL支持两种`primitive function type`, 即`quasi-affine expressions`和`quasipolynomials`. 
其中`quasi-affine expression`是通过参数空间或集合定义的, 由整数常数, 参数和集合变量, 加法, 减法和整数分割组成. 这里可以参考[零基础入门多面体编译中的仿射表达式](https://zhuanlan.zhihu.com/p/627312844).

在`isl`中, `quasi-affine`描述开始于结构化的`named integer tuple`模板, 其次是`->`符号, 然后使用方括号包裹着一个使用了前面变量的表达式, 最后整个表达式包含在花括号中. 如果`quasi-affine`表达式没有`domain space`, 则可以省略前面`named integer tuple`以及`->`符号. 如果涉及任何常量参数, 则在外部继续添加参数的表示. 比如:

$$
	[n] -> { [x] -> [2*floor((4 n + x)/9)] }
$$

其中`n`是参数,`x`是变量.

`quasipolynomials`是`quasi-affine expression`中的多项式表示. 也就是它还允许乘法, 但是不允许构建涉及乘法的表达式的整数划分

$$
  [n] -> { [x] -> (n*floor((4 n + x)/9)) }
$$

In [10]:
print("quasi-affine expression :")
print(isl.aff("[n] -> { [x] -> [2*floor((4 n + x)/9)] }"))
try:
  print(isl.aff("[n] -> { [x] -> [n*floor((4 n + x)/9)] }"))
except:
  print("quasi-affine expression only expect constant value multiply ")

quasi-affine expression :
[n] -> { [x] -> [(2*floor((4n + x)/9))] }
quasi-affine expression only expect constant value multiply 


syntax error (1, 20): expecting constant value
got keyword 'floor'


### Piecewise Quasi-Affine Expression

在`isl`中，分段`quasi-affine`表达式被写成一系列有条件的aff表达式，该序列由分号分离并包裹在花括号中:


In [11]:
print("Piecewise quasi-affine expression :")
aff = isl.pw_aff("[n] -> { [x] -> [2*floor((4 n + x)/9)] :  x > 10 ; [x] -> [2*floor((4 n + x)/9)] :  x < 3   }")
print(aff, "space:", aff.get_space())
aff = isl.pw_aff("[n] -> { [x] -> [x + 1] : x < n ; [x] -> [0] : x = n - 1 }")
print(aff, "space:", aff.get_space())

Piecewise quasi-affine expression :
[n] -> { [x] -> [(2*floor((4n + x)/9))] : x >= 11; [x] -> [(2*floor((4n + x)/9))] : x <= 2 } space: [n] -> { [x] -> [o0] }
[n] -> { [x] -> [(n)] : x = -1 + n; [x] -> [(1 + x)] : x <= -2 + n } space: [n] -> { [x] -> [o0] }


如果多个`aff`的`space`不同, 那么称为`union_pw_aff`:

In [12]:
aff = isl.union_pw_aff("{ [x] -> [x+1] : x < 10;  [x,y] -> [0] }")
print(aff, "space:", aff.get_space())
aff = isl.union_pw_aff("{ [x] -> [x+1] : x < 10;  [] -> [0] }")
print(aff, "space:", aff.get_space())

{ [x] -> [(1 + x)] : x <= 9; [x, y] -> [(0)] } space: {  :  }
{ [] -> [(0)]; [x] -> [(1 + x)] : x <= 9 } space: {  :  }


`aff` 也支持各种操作:
1. `sum`, 将`space`相同的表达式的`value`求和 (使用`add`函数).
2. `union`, 使用`union add`函数.
3. `pullback`, 函数`composition`, 将一个函数的输出作用到另一个函数的输入上.

In [13]:
print("aff sum", isl.aff("{ [x] -> [x + 2] }").add(isl.aff("{ [x] -> [x - 1] }")))
print("aff union", isl.union_pw_aff(
    "{ S_0[x] -> [x + 2] }").union_add(isl.aff("{ S_1[y] -> [y - 1] }")))

a1 = isl.multi_union_pw_aff("R[{ A[i,j] -> [i + j]; E[x] -> [-x] }]")
a2 = isl.union_pw_multi_aff("{ C[x] -> A[x,x]; D[x] -> E[2x] }")
print("aff pullback:", a1.pullback(a2))


aff sum { [x] -> [(1 + 2x)] }
aff union { S_1[y] -> [(-1 + y)]; S_0[x] -> [(2 + x)] }
aff pullback: R[{ D[x] -> [(-2x)]; C[x] -> [(2x)] }]


## Reductions

`Reductions`代表其基本表达式的最大或最小值。 `isl`定义的唯一`Reductions`类型是`isl_qpolynomial_fold`。 并且无法直接创建这个类型对象, 但是可以通过`isl_pw_qpolynomial_bound`函数来返回.

## Multiple Expressions

`Multiple Expressions`表示零或更多`base expression`组成的序列, 所有`base expression`都在同一`domain space`上定义. `Multiple Expressions`的`domain space`与`base expression`相同, 但是`range space`可以是任何. 如果基本表达式具有`set space`, 相应的多个表达式也具`set space`. `value`或`identifier`的类型对象没有关联的`space`. 因此, 多个`value`或多个`identifier`的`space`始终是`set space`. 同样, `multiple union piecewise affine expression`的空间始终`set space`. 如果基本表达式不是全部的, 则相应的零维`Multiple Expressions`可能具有一个显式`domain`, 该域可以跟踪任何`base expression`之外的域.

`multiple expression` 类型包含: `isl_multi_val`, `isl_multi_id`,`isl_multi_aff`, `isl_multi_pw_aff`, `isl_multi_union_pw_aff`

In [14]:
print("multi_id 构造:")
print(isl.multi_id("{ [A,B,C] }"))
print(isl.multi_id("[D, E] -> { [A,B,C] }"))
print("multi val 构造:")
print(isl.multi_val("{ [10,20,30] }"))
print(isl.multi_val("[A, B] -> { [10,20,30] }"))

multi_id 构造:
{ [A, B, C] }
[D, E] -> { [A, B, C] }
multi val 构造:
{ [10, 20, 30] }
[A, B] -> { [10, 20, 30] }


在`isl`中, `aff`的`tuple`被表示为`isl_multi_aff`, 他的写法与`aff`类似, 但是用`[]`括起来的`aff`表达式泛化为一个结构化的命名整数元组模板(`structured named integer tuple template`), 同时这个模板的变量被替换为依赖输入命名整数元组模板变量的`aff`表达式.

In [15]:
print("isl_multi_aff 构造:")
print(isl.multi_aff("{ [i] -> [[i] -> [i+1]] }"))
print(isl.multi_aff("{ [x, y] -> [[x-2,y+1] -> [x]] }"))
print(isl.multi_aff("{ [x, y] -> [(x), (x+1)] }"))

print("isl_multi_pw_aff 构造:")
print(isl.multi_pw_aff("{ [i] -> [(i + 1: i>0), (i - 1: i < 0)]  }"))
print(isl.multi_pw_aff("{ [i] -> [[i] -> [i-1 : i < 1]]  }"))
print(isl.multi_pw_aff("{ [i] -> [[i+2] -> [(i-1 : i < 1), (i-3 : i > 1) ]] }"))

print("isl_multi_union_pw_aff 构造:")
print(isl.multi_union_pw_aff("[{ S_1[m] -> [m + 10] :  m > 0 }, { S_2[n] -> [n-10] : n < 0 } ]"))

isl_multi_aff 构造:
{ [i] -> [[(i)] -> [(1 + i)]] }
{ [x, y] -> [[(-2 + x), (1 + y)] -> [(x)]] }
{ [x, y] -> [(x), (1 + x)] }
isl_multi_pw_aff 构造:
{ [i] -> [((1 + i) : i > 0), ((-1 + i) : i < 0)] }
{ [i] -> [[(i)] -> [((-1 + i) : i <= 0)]] }
{ [i] -> [[(2 + i)] -> [((-1 + i) : i <= 0), ((-3 + i) : i >= 2)]] }
isl_multi_union_pw_aff 构造:
[{ S_1[m] -> [(10 + m)] : m > 0 }, { S_2[n] -> [(-10 + n)] : n < 0 }]
